In [ ]:
# dl_day17은 휴가사용해서 수업을 못들음
# 금일은 gpt/bert 파인튜닝을 이용한 분류기를 만들 것
# 오후는 강사님이 프로젝트 진행 상황 체크할 것
# 추가로 앞으로의 일정을 함께 논의할 것
# 일정관리 필요
# 프로젝트 내에서 여러 문제에 대해서는 조별로 얘기를 나눌 것임

https://wikidocs.net/115055

In [1]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Bert-base의 토크나이저

In [2]:
result = tokenizer.tokenize('Here is the sentence I want embeddings for.')

In [4]:
result  # embedding이라는 단어가 없어서 쪼개서 임베딩 진행(subword)

['here',
 'is',
 'the',
 'sentence',
 'i',
 'want',
 'em',
 '##bed',
 '##ding',
 '##s',
 'for',
 '.']

In [5]:
print(tokenizer.vocab['here'])

2182


In [6]:
print(tokenizer.vocab['embeddgings']) # 없는 단어이므로 에러가 나와야 함
# sub word로 나눴을 때는 임베딩이 될 수 있었기 때문에 result에서는 정상적으로 실행된 것임

KeyError: 'embeddgings'

In [7]:
print(tokenizer.vocab['##ding']) # ##ding은 있는 단어임;; 근데 왜 있는 건지 모르겠음

4667


In [10]:
print(len(tokenizer.vocab))
tokenizer.vocab

30522


OrderedDict([('[PAD]', 0),
             ('[unused0]', 1),
             ('[unused1]', 2),
             ('[unused2]', 3),
             ('[unused3]', 4),
             ('[unused4]', 5),
             ('[unused5]', 6),
             ('[unused6]', 7),
             ('[unused7]', 8),
             ('[unused8]', 9),
             ('[unused9]', 10),
             ('[unused10]', 11),
             ('[unused11]', 12),
             ('[unused12]', 13),
             ('[unused13]', 14),
             ('[unused14]', 15),
             ('[unused15]', 16),
             ('[unused16]', 17),
             ('[unused17]', 18),
             ('[unused18]', 19),
             ('[unused19]', 20),
             ('[unused20]', 21),
             ('[unused21]', 22),
             ('[unused22]', 23),
             ('[unused23]', 24),
             ('[unused24]', 25),
             ('[unused25]', 26),
             ('[unused26]', 27),
             ('[unused27]', 28),
             ('[unused28]', 29),
             ('[unused29]', 30),
  

In [12]:
# BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt', 'w', encoding='utf-8') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')

In [13]:
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [ ]:
# bert 모델
# a라는 모델을 만들기 위해서 a라는 토큰나이저를 사용해서 만들어졌음
# 그럼 b라는 토큰에 대해서는 이해하지 못함
# ex bert토크나이저 : 바나나 - 20, 딸기 - 35, ...

# B라는 bert 모델, B라는 bert 모델을 만들 때 사용된 토크나이저
# ex bert토크나이저 : 바나나 - 40, 딸기 - 100, ...

# 이에 따라 bert 모델 종류가 많음

In [14]:
from transformers import TFBertForMaskedLM

In [15]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased') # 모델을 바꾸고 싶다면 여기에 작성된 bert를 바꾸면 됨
# masked 방식으로 학습된 BERT 모델 다운로드

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [18]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[ 101, 4715, 2003, 1037, 2428, 4569,  103, 1012,  102]])>, 'token_type_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [19]:
print(inputs['input_ids']) # 103은 예측 단어, 1012는 .이고 102는 end symbol같음

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [22]:
print(inputs['token_type_ids']) # 같은 문장인지 다른 문장인지 구분하는 세그먼트 인코딩 결과를 확인할 수 있음

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


In [23]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [24]:
pip('Soccer is a really fun [MASK].') # mask에 들어갈 수 있는 후보 5개
# 결과는 리스트 안에 dict형태로 출력해서 보여줌

[{'score': 0.7621126770973206,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20341919362545013,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208515778183937,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.0018630229169502854,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.001335482462309301,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [25]:
pip('The Avengers is a really fun [MASK].')

[{'score': 0.25628939270973206,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.1728411763906479,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107713729143143,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07248974591493607,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046648114919662,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [ ]:
# https://huggingface.co/google-bert/bert-base-chinese  # klue/bert-base 검색

In [26]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [27]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [28]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

In [29]:
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [30]:
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [31]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963505625724792,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.0259577464312315,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033931583166122,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924369536340237,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844232954084873,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [32]:
pip('오늘은 비가 많이 [MASK]다.')

[{'score': 0.333875834941864,
  'token': 5740,
  'token_str': '내렸',
  'sequence': '오늘은 비가 많이 내렸 다.'},
 {'score': 0.26037460565567017,
  'token': 1458,
  'token_str': '왔',
  'sequence': '오늘은 비가 많이 왔 다.'},
 {'score': 0.11017513275146484,
  'token': 1443,
  'token_str': '오',
  'sequence': '오늘은 비가 많이 오 다.'},
 {'score': 0.08552484214305878,
  'token': 1445,
  'token_str': '온',
  'sequence': '오늘은 비가 많이 온 다.'},
 {'score': 0.03042539395391941,
  'token': 4848,
  'token_str': '내리',
  'sequence': '오늘은 비가 많이 내리 다.'}]

In [33]:
pip('자연어 처리는 딥러닝의 한 [MASK]다.')

[{'score': 0.6545459032058716,
  'token': 3993,
  'token_str': '분야',
  'sequence': '자연어 처리는 딥러닝의 한 분야 다.'},
 {'score': 0.0668841227889061,
  'token': 4137,
  'token_str': '단계',
  'sequence': '자연어 처리는 딥러닝의 한 단계 다.'},
 {'score': 0.0646013617515564,
  'token': 4787,
  'token_str': '요소',
  'sequence': '자연어 처리는 딥러닝의 한 요소 다.'},
 {'score': 0.061697449535131454,
  'token': 5285,
  'token_str': '종류',
  'sequence': '자연어 처리는 딥러닝의 한 종류 다.'},
 {'score': 0.04614029452204704,
  'token': 4337,
  'token_str': '형태',
  'sequence': '자연어 처리는 딥러닝의 한 형태 다.'}]

In [34]:
pip('떠오르는 아이디어를 잘 정리하여 [MASK]를 등록했다.') # 원하는 단어는 '특허'였음
# 위와 같은 경우 비문이 많음. 문법에 맞지 않는 것이 꽤 등장하기 때문에 이를 다듬는 작업도 필요함

[{'score': 0.14783203601837158,
  'token': 8958,
  'token_str': '블로그',
  'sequence': '떠오르는 아이디어를 잘 정리하여 블로그 를 등록했다.'},
 {'score': 0.04735827445983887,
  'token': 5299,
  'token_str': '홈페이지',
  'sequence': '떠오르는 아이디어를 잘 정리하여 홈페이지 를 등록했다.'},
 {'score': 0.04236149787902832,
  'token': 4979,
  'token_str': '프로젝트',
  'sequence': '떠오르는 아이디어를 잘 정리하여 프로젝트 를 등록했다.'},
 {'score': 0.04110949859023094,
  'token': 16,
  'token_str': ',',
  'sequence': '떠오르는 아이디어를 잘 정리하여, 를 등록했다.'},
 {'score': 0.0296604223549366,
  'token': 6691,
  'token_str': 'SNS',
  'sequence': '떠오르는 아이디어를 잘 정리하여 SNS 를 등록했다.'}]

In [35]:
# 자연어 예측은 2가지가 있는데
# MASK 방식, NSP(Next sentence prediction) 방식이 있음
# MASKED 방식은 위에서 진행한 방식
# NSP는 다음 문장 예측 방식임

In [36]:
from transformers import TFBertForNextSentencePrediction

In [37]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [38]:
from transformers import AutoTokenizer

In [39]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# 0 : 서로 이어지는 문장
# 1 : 상관없는 문장

In [40]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

In [41]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

In [42]:
print(encoding['input_ids'])  # 출력에서 Tensor는 다차원 배열을 의미한다고 보면 됨
# 출력에서 101은 시작하는 토큰(CLS 토큰이라고 함), 1999부턴 문장, 102는 마치는 토큰(SEP토큰)

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


In [44]:
encoding['input_ids'][0]

<tf.Tensor: shape=(58,), dtype=int32, numpy=
array([  101,  1999,  3304,  1010, 10733,  2366,  1999,  5337, 10906,
        1010,  2107,  2004,  2012,  1037,  4825,  1010,  2003,  3591,
        4895, 14540,  6610,  2094,  1012,   102, 10733,  2003,  8828,
        2007,  1996,  2224,  1997,  1037,  5442,  1998,  9292,  1012,
        1999, 10017, 10906,  1010,  2174,  1010,  2009,  2003,  3013,
        2046, 17632,  2015,  2000,  2022,  8828,  2096,  2218,  1999,
        1996,  2192,  1012,   102])>

In [45]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


In [46]:
print(encoding['token_type_ids']) # 0이 첫 번째 문장, 1이 두 번째 문장
# 여러 문장이 들어가면 CLS, SEP가 들어가게 됨

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


In [47]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [48]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 6.395771, -6.376442]], dtype=float32)>

In [49]:
import tensorflow as tf

In [50]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs) # logits의 값을 확률로 변환

tf.Tensor([[9.9999714e-01 2.8385537e-06]], shape=(1, 2), dtype=float32)


In [59]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy()) # 즉 두 문장이 이어지는 문장이라고 판단

최종 예측 레이블 : [0]


In [60]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [61]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[1.2580943e-04 9.9987423e-01]], shape=(1, 2), dtype=float32)


In [62]:
tf.math.argmax(probs, axis=-1).numpy() # 관련없는 내용으로 판단

array([1], dtype=int64)

In [63]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [64]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [70]:
# 이어지는 두 개의 문장
# prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
# next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
# prompt = "저녁에는 된장찌개를 먹을 거야"
# next_sentence = "배고프다. 점심에는 뭘 먹지?"
prompt = "비 오는 날에는 막걸리가 최고지."
next_sentence = "파전이 맛있어"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [71]:
# 코랩에서 TPU 사용 방법

In [74]:
# import tensorflow as tf
# import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.TPUStrategy(resolver)


# def create_model():
#   return tf.keras.Sequential(
#       [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
#        tf.keras.layers.Conv2D(256, 3, activation='relu'),
#        tf.keras.layers.Flatten(),
#        tf.keras.layers.Dense(256, activation='relu'),
#        tf.keras.layers.Dense(128, activation='relu'),
#        tf.keras.layers.Dense(10)])
# with strategy.scope():
#   model = create_model()
#   model.compile(optimizer='adam',
#                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                 metrics=['sparse_categorical_accuracy'])

# KorBERT를 이용한 네이버 영화 리뷰 분류
현재 코랩에서 잘 안됨;

In [75]:
# korbert (ETRI)
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [76]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x2a993897730>)

In [77]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [79]:
print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# fine tuning
# 한국어 버전의 구글 bert 모델 -> 파인튜닝 -> ETRI라는 기관에서 korbert를 만듦 -> 배포

In [80]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.reset_index(drop=True)
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [81]:
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
test_data = test_data.reset_index(drop=True)
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [82]:
print(len(train_data))
print(len(test_data))

149995
49997


In [83]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [84]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [85]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
# 보 라는 단어가 vocab에 있었나봄

['보', '##는', '##내', '##내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [86]:
print(tokenizer.tokenize("홟는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
# 홟이라는 단어가 학습되진 않은듯

['[UNK]', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [87]:
for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

[ C L S ]
보
# # 는
# # 내
# # 내
그 대 로
들 어 맞
# # 는
예 측
카 리 스 마
없
# # 는
악 역
[ S E P ]


In [88]:
for elem in tokenizer.encode("happy birthday~!"):
  print(tokenizer.decode(elem))
#  한국어 bert이므로 영문 단어가 존재하지 않게 되어버림 - happy, birth, day, ~, ! 이렇게 돼야함

[ C L S ]
h a
# # p p
# # y
b
# # i r
# # t h
# # d a y
~
!
[ S E P ]


In [89]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)

[CLS] : 2


In [90]:
tokenizer.decode(tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [94]:
max_seq_len = 128
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길이 :', len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이 : 128


c:\Users\fkfma\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [95]:
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [96]:
# 마스크 인풋
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])
# 토큰 길이만큼 1로 채움

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [99]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [100]:
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 149995/149995 [00:33<00:00, 4451.29it/s]


In [101]:
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 49997/49997 [00:10<00:00, 4621.24it/s]


In [102]:
# 최대 길이: 128
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
# 파인튜닝 이전에 똑같은 구조로 구성해줘야 학습이 가능하게 됨
# 이미 사전학습된 모델에 파인튜닝을 진행

In [103]:
model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [104]:
max_seq_len = 128

In [105]:
# 해당 셀의 코드는 코랩에서 작동 안함
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [106]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
import tensorflow as tf
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=2, batch_size=64, validation_split=0.2)